In [128]:
import pickle
import pandas as pd
import re
import numpy as np
import os
from IPython.display import display, Audio
from pathlib import Path
import sys
sys.path.insert(1, '../modules')
import data

# Creating manual audio annotations csv

In [2]:
with open('/green-projects/project-sonyc_redhook/workspace/share/redhook-analysis/output/annotation_list.pickle', "rb") as f:
       annotation_list = pickle.load(f)

In [4]:
annotation_list[0]

(1573060932.0, 'n')

In [15]:
annotation_list_filename = [(str(annotation[0]).split('.')[0]+'.wav', annotation[1]) for annotation in annotation_list]

In [19]:
audio_annotations = pd.DataFrame.from_records(annotation_list_filename, columns=['filename', 'annotation'])
audio_annotations.tail()

,filename,annotation
1533,1573852358.wav,n
1534,1573852939.wav,n
1535,1573848606.wav,n
1536,1573848526.wav,n
1537,1573853688.wav,n


In [119]:
audio_annotations.to_csv\
('/green-projects/project-sonyc_redhook/workspace/share/redhook-analysis/output/audio_annotations.csv')

# Creating video annotations csv

In [20]:
with open('/green-projects/project-sonyc_redhook/workspace/share/video_info.pickle', "rb") as f:
       video_info = pickle.load(f)

In [27]:
next(iter(video_info))

'/home/cusp/cmydlarz/redhook_video/data/1573276672.ts'

In [99]:
video_annotations = []
for filename in video_info:
    start_timestamp = int(re.findall(r'[0-9]+', filename)[0])
    for frame in video_info[filename]:
        frame_offset = round(int(frame)/15)
        actual_timestamp = int(start_timestamp) + frame_offset
        info_arr = np.swapaxes(np.asarray(video_info[filename][frame]), 0, 1)
        #rearranging axes so that there's one class instance in every row
        info_arr_timestamps = np.insert(info_arr, 0, actual_timestamp, axis=1)
        info_arr_timestamps = np.insert(info_arr_timestamps, 0, frame_offset, axis=1)
        info_arr_timestamps = np.insert(info_arr_timestamps, 0, start_timestamp, axis=1)
#         print(info_arr_timestamps)
        for entry in info_arr_timestamps:
            video_annotations.append(entry)      

In [101]:
video_annotations = pd.DataFrame.from_records(video_annotations, \
columns=['start timestamp', 'frame offset', 'actual timestamp', 'bounding box', 'probability', 'class'])

In [125]:
video_annotations.tail()

,start timestamp,frame offset,actual timestamp,bounding box,probability,class
2231005,1573303753,6,1573303759,"[209.0, 219.0, 422, 408]",0.613638,truck
2231006,1573303753,6,1573303759,"[17.0, 335.5, 204, 123]",0.886392,car
2231007,1573303753,6,1573303759,"[372.5, 221.5, 271, 411]",0.753392,truck
2231008,1573303753,6,1573303759,"[16.0, 336.0, 208, 122]",0.877356,car
2231009,1573303753,6,1573303759,"[536.5, 217.0, 115, 388]",0.460074,truck


In [104]:
def search_and_display_audio(timestamp):
    audio_file = (sorted(Path('/green-projects/project-sonyc_redhook/workspace/share/truck_audio/redhook_truck_audio/10s/')\
           .rglob(timestamp+'.wav')) + \
         sorted(Path('/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/')\
           .rglob(timestamp+'*.wav')))
    print(timestamp)
    display(Audio(filename=audio_file[0]))

In [222]:
audio_files = []
for video_timestamp in video_annotations['start timestamp']:
    #get calendar date of video timestamp
    year = str(data.convert_timestamps(float(video_timestamp)).year)
    month = str(data.convert_timestamps(float(video_timestamp)).month)
    day = str(data.convert_timestamps(float(video_timestamp)).day)
    if int(day) < 10:  
        day = '0' + day
    calendar_date = year + '-' + month + '-' + day
    #only search in folder matching calendar_date
    matching_date_files = (sorted(Path('/green-projects/project-sonyc_redhook/workspace/share/truck_audio/redhook_truck_audio/10s/'+calendar_date)\
               .glob('.wav')) + \
             sorted(Path('/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/'+calendar_date)\
               .glob('*.wav')))
    found_file_match = False
    for file in matching_date_files:
        #get numerical timestamp of audio from file name
        audio_timestamp = int(re.findall('\d{5,}', str(file))[0])
        #if the diff between audio and video timestamp is less than 10 sec
        #NOTE: sometimes there's more than one matching audio file for the video file? a bit confusing
        if abs(video_timestamp - audio_timestamp) < 10 and not found_file_match:
            found_file_match = True #we found a match!
            audio_files.append(file)
    if not found_file_match:
        #if no match, say so
        audio_files.append('n/a')

In [223]:
print(len(audio_files))
print(count)

2231010
41381


In [224]:
video_annotations['matching audio file'] = audio_files

In [228]:
audio_annotations.to_csv\
('/green-projects/project-sonyc_redhook/workspace/share/redhook-analysis/output/video_annotations.csv')

In [201]:
test = '/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/2019-11-16/1573930010.67.wav'
# re.findall('\d/(.*)$', test)
re.findall('\d{5,}', test)

['1573930010']

In [156]:
test = '1573930010.67.wav'
re.findall('^\d+', test)

['1573930010']

In [208]:
test = '2019-11-16'
test_file = (sorted(Path('/green-projects/project-sonyc_redhook/workspace/share/truck_audio/redhook_truck_audio/10s/'+test)\
               .glob('.wav')) + \
             sorted(Path('/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/'+test)\
               .glob('*.wav')))
for x in test_file:
    print(int(re.findall('\d{5,}', str(x))[0]))

1573880844
1573880845
1573882472
1573882528
1573883005
1573883349
1573884878
1573884938
1573887412
1573888594
1573891213
1573899217
1573902710
1573902770
1573909409
1573909489
1573911535
1573911878
1573911878
1573912787
1573912788
1573913659
1573915370
1573915569
1573915787
1573916100
1573916259
1573916279
1573916374
1573916377
1573916379
1573916900
1573917162
1573917812
1573918490
1573918490
1573918951
1573919739
1573920303
1573920510
1573920650
1573920879
1573921331
1573921818
1573922011
1573922496
1573923775
1573924074
1573924170
1573924731
1573924827
1573924890
1573925141
1573925994
1573926154
1573926171
1573926268
1573926755
1573926847
1573926908
1573927666
1573927830
1573927855
1573928123
1573928199
1573928311
1573928329
1573929796
1573929798
1573929812
1573929913
1573930010
1573930060
1573930218
1573930220
1573930816
1573931035
1573931410
1573931847
1573932350
1573932807
1573932905
1573932906
1573933018
1573933036
1573933151
1573933449
1573933721
1573933756
1573933907
1573934310

In [ ]:
#maybe from video file, get day, then only search in folders of that day

In [ ]:
rounded_audio_timestamps

In [126]:
#go through every audio file
#round the timestamps to the nearest 10 seconds, store in a dictionary with rounding as the key
for subdir, dirs, files in os.walk('/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/'):
    for file in files:
        print(os.path.join(subdir, file))


/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/2019-11-19/1574205098.47.wav
/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/2019-11-19/1574190679.2.wav
/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/2019-11-19/1574173579.67.wav
/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/2019-11-19/1574207686.2.wav
/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/2019-11-19/1574171330.33.wav
/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/2019-11-19/1574205380.4.wav
/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/2019-11-19/1574168233.27.wav
/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/2019-11-19/1574201161.47.wav
/green-projects/project-sonyc_redhook/workspace/share/redhook_truck_audio/10s/2019-11-19/1574184662.73.wav
/green-projects/project-sonyc_redhook/wo